In [6]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator
import collections
%matplotlib inline

#Getting the data
con = sqlite3.connect('tweetsSpring.db')
data = pd.read_sql("SELECT * from tweets where month = 5 and day != 26",con)
dataLast = pd.read_sql("SELECT * from tweets where month = 5 and day = 26",con)

In [11]:
#Progress
from sys import stdout
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [13]:
import json
import re
import pymorphy2
import Stemmer

class TweetTextParser():

    def __init__(self):
        print 'Invoking Processor...'
        self.morph = pymorphy2.MorphAnalyzer()
        self.stemmer = Stemmer.Stemmer('russian')
        try:
            self.emo_db = json.load(open('pyalchemy/emoji_database','r'))
        except:
            print('No emoji database found')


    def processContents(self, myText):
        myText = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?¬´¬ª‚Äú‚Äù‚Äò‚Äô]))', '', myText)
        words = [word for word in re.findall(r'(?u)[@|#]?\w+', myText) if not word.startswith(('@','#'))]
        words = self.stemmer.stemWords(words)
        return words
        
    def resolveEmoji(self, myText):
        emostr = []
        emo_db = self.emo_db
        b = myText.encode('unicode_escape').split('\\')
        c = [point.replace('000','+').upper() for point in b if len(point) > 8 and point[0] == 'U']
        [emostr.append(emo_db[emo[:7]]) for emo in c if emo[:7] in emo_db]
        return myText

textProcessor = TweetTextParser()

Invoking Processor...


In [14]:
from time import time
morph = pymorphy2.MorphAnalyzer()
stemmer = Stemmer.Stemmer('russian')

#Cleaning up the data
t0 = time()
terms = []
n = len(data.index)
#n = 100000
for i in range(n):
    terms.append(textProcessor.processContents(data.content_lower[i]))
print time()-t0

31.3677601814


In [15]:
data['terms'] = terms[:]

In [10]:
print data.created_at.tail
print dataLast.created_at.tail

<bound method Series.tail of 0         2015-05-01 00:00:33
1         2015-05-01 00:00:41
2         2015-05-01 00:00:46
3         2015-05-01 00:01:00
4         2015-05-01 00:01:07
5         2015-05-01 00:01:10
6         2015-05-01 00:01:21
7         2015-05-01 00:01:28
8         2015-05-01 00:01:44
9         2015-05-01 00:02:01
10        2015-05-01 00:02:09
11        2015-05-01 00:02:13
12        2015-05-01 00:02:16
13        2015-05-01 00:02:51
14        2015-05-01 00:02:59
15        2015-05-01 00:03:08
16        2015-05-01 00:03:24
17        2015-05-01 00:03:32
18        2015-05-01 00:03:50
19        2015-05-01 00:03:57
20        2015-05-01 00:04:03
21        2015-05-01 00:04:23
22        2015-05-01 00:04:30
23        2015-05-01 00:04:31
24        2015-05-01 00:04:34
25        2015-05-01 00:04:35
26        2015-05-01 00:04:41
27        2015-05-01 00:04:57
28        2015-05-01 00:05:01
29        2015-05-01 00:05:03
                 ...         
667468    2015-05-25 23:56:09
667469    2

In [16]:
for row in data.index[:10]: 
    for word in data['terms'][row]: print word

–ù—É
–∫—Ç–æ
–µ—â
–ø–æ–ø–∞–¥–µ—Ç
–Ω–∞
–∏–Ω–≤–µ–Ω—Ç–∞—Ä–∏–∑–∞—Ü
–µ—Å–ª
–Ω–µ
—è
–ø–æ–∂–∞–ª—É–π—Å—Ç
–ø—É—Å—Ç
—É—Ç—Ä
–≤—Å–µ
—ç—Ç
–æ–∫–∞–∂–µ—Ç
–¥—É—Ä–Ω
—Å–Ω–æ–º
–°–º–æ—Ç—Ä
—è
–≤
–ú–æ—Å–∫–≤
–°–µ–π—á–∞—Å
9
—è—Å–Ω
–£—Ç—Ä–æ–º
7
–æ–±–ª–∞—á–Ω
–Ω–µ–±–æ–ª—å—à
–¥–æ–∂–¥
–î–Ω–µ–º
11
–ø–∞—Å–º—É—Ä–Ω
–¥–æ–∂–¥
–í–µ—á–µ—Ä
–ê–∞
—Ç—ã
–≤
–ú–æ—Å–∫–≤
–ù–µ–º–Ω
–ø—Ä–∏—è—Ç–Ω
–º–æ–º–µ–Ω—Ç
–ù—É
–≤–∑—è–≤
—Ç–≤–æ
—Ä—É—á–∫
–≤
—Å–≤–æ
–Ω–µ—Å–∏–ª—å–Ω
—Å–∂–∏–º–∞
–¥–∞
–¥–∞
—è
–≤
–ú–æ—Å–∫–≤
–†–æ—Å—Å
–∞—Ö–∞—Ö—Ö–∞—Ö
–∞—Ä–≥—É–º–µ–Ω—Ç–Ω
–∞—Ä–≥—É–º–µ–Ω—Ç


In [22]:
from collections import Counter
import nltk

words = []
for i in range(n):
    words += terms[i]

fdist = nltk.FreqDist(words)
sortedDist = sorted(fdist.items(), key=operator.itemgetter(1),reverse=True)

sortedDist = [x for x in sortedDist if len(x[0]) > 2]
#allTheWords = [x for x in moreThan3 if x[1] > 1]
interestingVocab = [x[0] for x in sortedDist]
print 'Vocab Length: ', len(interestingVocab)

Vocab Length:  262391


In [23]:
for i in range(10):
    print interestingVocab[i]

—á—Ç–æ
—Ç–∞–∫
–∫–∞–∫
–ú–æ—Å–∫–≤
–º–µ–Ω
–≤—Å–µ
–º–Ω–µ
–±—ã–ª
—Ç–µ–±
–ø—Ä–æ—Å—Ç


In [24]:
#Find TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

trainingList = []
for i in range(n):
    trainingList.append(' '.join(data['terms'][i]))

tfidf_vectorizer = TfidfVectorizer(vocabulary = interestingVocab)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(trainingList)  #finds the tfidf score with normalization

In [31]:
import time
def findSimilarTweets(queryTweet, data, threshhold):
    print 'Query:', queryTweet
    processedTweet = ' '.join(textProcessor.processContents(queryTweet))
    
    queryTweetRepresentation = tfidf_vectorizer.transform([processedTweet])
    start_time = time.time()
    cosine_similarities = cosine_similarity(queryTweetRepresentation, tfidf_matrix_train)[0]
    nearest_objects = cosine_similarities[cosine_similarities>0.4]
    indices = nearest_objects.argsort()[::-1]
    elapsed_time = time.time() - start_time
    print 'time: ', elapsed_time
    
    print 'cosine scores ==>', nearest_objects[indices]
    print ''
    print 'Results:'
    print data[indices]
    
    return indices

In [32]:
#before
inidiciesOfTweets = findSimilarTweets(u'–≤ –º–µ—Ç—Ä–æ –æ–±—ä—è–≤–ª—è—é—Ç —Å—Ç–∞–Ω—Ü–∏–∏',data['content'])

Query: –≤ –º–µ—Ç—Ä–æ –æ–±—ä—è–≤–ª—è—é—Ç —Å—Ç–∞–Ω—Ü–∏–∏
time:  0.217649936676
cosine scores ==> [ 0.75964594  0.73711167  0.71805737  0.71462365  0.69707083  0.69212216
  0.68498465  0.6784575   0.67471516  0.6738558   0.67147243  0.66798734
  0.66493821  0.65294042  0.6516734   0.64886296  0.6468789   0.63383193
  0.63178933  0.6277312   0.62761031  0.62701111  0.62164232  0.61345563
  0.61131433  0.59158023  0.59102898  0.58514276  0.58495025  0.58495025
  0.58495025  0.58495025  0.58495025  0.58495025  0.58495025  0.58495025
  0.58495025  0.58495025  0.58495025  0.58495025  0.57897889  0.57746253
  0.57470785  0.57227371  0.56643359  0.5628667   0.56256038  0.56235789
  0.55884245  0.55828391  0.55544978  0.55544978  0.55544978  0.55544978
  0.54969781  0.54801253  0.54638102  0.54635907  0.54594384  0.54500748
  0.54233213  0.54202074  0.54202074  0.54202074  0.54202074  0.54202074
  0.54202074  0.54202074  0.53763038  0.53629846  0.53614752  0.535267
  0.52784001  0.52716108  0.525

In [ ]:
len(data)

In [74]:
print 'Vocab Length: ', len(interestingVocab)

Vocab Length:  15268


In [75]:
print data['cleanText'][0]

–†–µ–±—è—Ç–∞, –º—ã –≤—Å–µ–º —Ä–∞–¥—ã!


In [53]:
len(trainigList)

673685

In [2]:
from scipy.sparse import csr_matrix

print('hi')

hi


In [45]:
from sparselsh import LSH
from scipy.sparse import csr_matrix

t0 = time()
lsh = LSH( 100,
           tfidf_matrix_train.shape[1],
           num_hashtables=1,
           storage_config={"dict":None})
print time()-t0

NameError: name 'tfidf_matrix_train' is not defined

In [ ]:
for ix in xrange(tfidf_matrix_train.shape[0]):
    x = tfidf_matrix_train.getrow(ix)
    lsh.index(x)
    progress(ix,tfidf_matrix_train.shape[0])

6.470348%

In [ ]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

# Dimension of our vector space
dimension = 478524

# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 1000)

# Create engine with pipeline configuration
engine = Engine(dimension, lshashes=[rbp])

# Index 1000000 random vectors (set their data to a unique string)
for index in range(673685):
    v = tfidf_matrix_train[index:index+1]
    engine.store_vector(v, 'data_%d' % index)
    progress(index,673685)

# Create random query vector
query = numpy.random.randn(dimension)

# Get nearest neighbours
N = engine.neighbours(query)